In [1]:
%load_ext autoreload
%autoreload 2

In [24]:
import os
import pickle
from src.paths import LOCAL_MODELS_PATH
from src.processing.text_preprocessing import DialogueEnricher

from src.infering import EntityRelationInferer
from src.processing.dialogre_processing import DialogREDatasetTransformer
from src.paths import LOCAL_MODELS_PATH, LOCAL_PROCESSED_DATA_PATH, LOCAL_RAW_DATA_PATH

dialogue_list = [
    "Speaker 1: A new place for a new Ross. I'm gonna have you and all the guys from work over once it's y'know, furnished.",
    "Speaker 2: I must say it's nice to see you back on your feet.",
    "Speaker 1: Well I am that. And that whole rage thing is definitely behind me.",
    "Speaker 2: I wonder if its time for you to rejoin our team at the museum?",
    "Speaker 1: Oh Donald that-that would be great. I am totally ready to come back to work. I—What? No! Wh… What are you doing?!!  GET OFF MY SISTER!!!!!!!!!!!!!"
    ]
relation = {
    "y": "museum",
    "x": "Speaker 1",
    "rid": [
    28
    ],
    "r": [
    "per:place_of_work"
    ],
    "t": [
    "rejoin"
    ],
    "x_type": "PER",
    "y_type": "STRING"
}

ent_x, ent_y = relation['x'], relation['y']

T2 = 0.32
relation_type_count = 36
bert_config_file = LOCAL_MODELS_PATH / "downloaded/bert-base/bert_config.json"
vocab_file = LOCAL_MODELS_PATH / "downloaded/bert-base/vocab.txt"
model_path=LOCAL_MODELS_PATH / "fine-tuned/dialogre-fine-tuned/bert_base/model_best.pt"
relation_label_dict = LOCAL_RAW_DATA_PATH / 'dialog-re/relation_label_dict.json'

inferer = EntityRelationInferer(
    bert_config_file = bert_config_file, 
    vocab_file = vocab_file, 
    model_path = model_path, 
    relation_type_count = relation_type_count, 
    relation_label_dict = relation_label_dict,
    T2 = T2)

rid_prediction, relation_label = inferer.infer_relations(' '.join(dialogue_list), ent_x, ent_y)

model_path = LOCAL_MODELS_PATH / 'custom/relation-identification/xgboost/dialog-re-binary-enriched'

def load_model(path):
    model = pickle.load(open(os.path.join(path, 'model.pkl'), 'rb'))
    le_dict = pickle.load(open(os.path.join(path, 'label_encoder_dict.pkl'), 'rb'))
    vectorizer = pickle.load(open(os.path.join(path, 'vectorizer.pkl'), 'rb'))
    scaler = pickle.load(open(os.path.join(path, 'scaler.pkl'), 'rb'))
    return model, le_dict, vectorizer, scaler

enricher = DialogueEnricher()

model, le_dict, vectorizer, scaler = load_model(model_path)


le_dict

RuntimeError: Error(s) in loading state_dict for BertForSequenceClassification:
	Missing key(s) in state_dict: "classifier.0.weight", "classifier.0.bias". 
	Unexpected key(s) in state_dict: "classifier.weight", "classifier.bias". 

In [22]:
enriched_dialogues = enricher.enrich(dialogues)
enriched_dialogues

100%|██████████| 1/1 [00:00<00:00,  8.20it/s]


[(['Speaker 1: Hey!',
   'Speaker 2: Hey.',
   "Speaker 3: Hey, man. What's up?",
   "Speaker 1: Maybe you can tell me. My agent would like to know why I didn't show up at the audition I didn't know I had today. The first good thing she gets me in weeks. How could you not give me the message?!",
   "Speaker 3: Well, I'll tell ya I do enjoy guilt, but, ah, it wasn't me.",
   'Speaker 2: Yes, it was! It was him! Uh huh! Okay, it was me!',
   'Speaker 1: How is it you?',
   "Speaker 2: Well, it was just, it was all so crazy, you know. I mean, Chandler was in the closet, counting to 10, and he was up to 7 and I hadn't found a place to hide yet. I-I-I meant to tell you, and I wrote it all down on my hand. See, all of it.",
   "Speaker 1: Yep, that's my audition.",
   'Speaker 4: See, now this is why I keep notepads everywhere.',
   "Speaker 2: Yep, and that's why we don't invite you to play.",
   'Speaker 5: What is the great tragedy here? You go get yourself another appointment.',
   'Spea

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\muril\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
100%|██████████| 1/1 [00:00<00:00,  8.71it/s]


In [20]:
enriched_dialogues[0][1]

[{'y': 'casting director',
  'x': 'Ann',
  'x_type': 'PER',
  'y_type': 'STRING',
  'x_token_span': (269, 313),
  'y_token_span': (391, 392),
  'x_char_span': (966, 1154),
  'y_char_span': (1430, 1433),
  'min_words_distance': 81,
  'min_words_distance_pct': 0.038553069966682535,
  'spacy_features': {'x_pos': 'VERB',
   'x_dep': 'amod',
   'x_tag': 'VBG',
   'y_pos': 'PROPN',
   'y_dep': 'npadvmod',
   'y_tag': 'NNP'},
  'min_turn_distance': 3,
  'min_turn_distance_pct': 0.0014278914802475012}]

In [8]:
vectorizer

TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [9]:
scaler